## Modeling an idealized tree crown surface
David Diaz

A flexible geometric form, a generalized ellipsoid can be used to approximate the three-dimensional crown surface of a tree. A two-dimensional ellipsoid for modeling crowns was originally proposed by Horn (1971):

$\frac{x^E}{a^E}+\frac{y^E}{b^E}=1$

A three-dimensional variant has been attributed to Pollock (1996), although a similar model was also described by Koop (1989). The figure below was produced by Sheng et al. (2001) and refers to this form as the Pollock model.

<img src='./images/Sheng et al 2001_Figure1.PNG' width='600' align='center'>

In [1]:
import numpy as np
import ipyvolume.pylab as p3
from ipywidgets import (FloatSlider, IntSlider, 
                        VBox, interactive_output, 
                        Layout, Label)
from IPython.display import display

## A function for calculating crown surface points and visualizing them

In [6]:
def make_crown(crown_ratio, cr, cc):
    top_ht = 100
    ch = top_ht * crown_ratio
    bh = top_ht - ch
    xt, yt, zt = 0, 0, top_ht
    
    r = np.linspace(-cr,cr,25) # radii to generate surface valeus for
    theta = np.linspace(0,2*np.pi,32) # angles
    radius_matrix, theta_matrix = np.meshgrid(r,theta) # points within the crown
    x = radius_matrix * np.cos(theta_matrix) # x points within the crown
    y = radius_matrix * np.sin(theta_matrix) # y points within the crown
    
    # generate surface points of the crown
    z = ((1-(((x-xt)**2 + (y-yt)**2)**(cc/2))/(cr**cc))*(ch**cc) - ch + zt)**(1/cc) + bh
    
    p3.clear() # there seems to be a failure to clear the previous values from display
    
    # by default, ipyvolume shows Y-axis as up-down, and this can't apparently be modified
    # so switch out y and z in the plotting call
    p3.plot_wireframe(x,z,y, color='green') 
    p3.zlim(-50,50)
    p3.xlim(-50,50)
    p3.ylim(0,100)
    p3.style.use('nobox')
    return p3.show()

## Create some user interface widgets

In [9]:
ratio = FloatSlider(min=0.0,max=1.0,step=0.01,value=0.75,
                    description='Crown Ratio:',
                    continuous_update=False,
                    layout=Layout(width='50%'))
curvature = FloatSlider(min=0,max=5,step=0.01, value=1.5,
                        description='Curvature:',
                        continuous_update=False,
                        layout=Layout(width='50%'))
radius = IntSlider(min=0,max=50,step=1,value=15,
                     description='Crown Radius:',
                     continuous_update=False,
                     layout=Layout(width='50%'),
                     style={'description_width': 'initial'})

# lay out the widgets
title = Label('Tree Crown Modeler')
ui = VBox([title,ratio,radius,curvature])

## Connect the widgets to our crown function

In [10]:
output = interactive_output(make_crown,
                            {'crown_ratio': ratio,
                             'cr': radius,
                             'cc': curvature})

## And display it!

In [11]:
display(ui, output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'VBox(children=(Figure(camera_center=[0…